# System Design Interview Problems

This notebook contains object-oriented design problems and system design challenges commonly asked in technical interviews.

## Table of Contents
1. [Design Patterns](#design-patterns)
2. [OOP Design](#oop-design)
3. [System Components](#system-components)
4. [Scalability & Performance](#scalability)

<a id='design-patterns'></a>
## 1. Design Patterns

### Problem 1: Observer Pattern (Medium)

**Problem Statement:**
Implement the Observer pattern for a stock price monitoring system.

**Requirements:**
- `Subject` (Stock) maintains list of observers
- Observers get notified when stock price changes
- Support adding/removing observers
- Observers can be different types (email, SMS, logger)

**Use Case:**
Multiple systems need to react to stock price changes (alerts, charts, trading bots).

In [ ]:
from abc import ABC, abstractmethod
from typing import List

# Observer interface
class Observer(ABC):
    """Abstract base class for observers"""
    
    @abstractmethod
    def update(self, subject: 'Subject') -> None:
        """Called when subject's state changes"""
        pass

# Subject interface
class Subject(ABC):
    """Abstract base class for subjects"""
    
    @abstractmethod
    def attach(self, observer: Observer) -> None:
        """Attach an observer"""
        pass
    
    @abstractmethod
    def detach(self, observer: Observer) -> None:
        """Detach an observer"""
        pass
    
    @abstractmethod
    def notify(self) -> None:
        """Notify all observers"""
        pass

# Concrete Subject
class Stock(Subject):
    """Stock that observers can watch"""
    
    def __init__(self, symbol: str, price: float):
        self.symbol = symbol
        self._price = price
        self._observers: List[Observer] = []
    
    @property
    def price(self) -> float:
        return self._price
    
    @price.setter
    def price(self, value: float) -> None:
        """When price changes, notify all observers"""
        if value != self._price:
            self._price = value
            self.notify()
    
    def attach(self, observer: Observer) -> None:
        if observer not in self._observers:
            self._observers.append(observer)
    
    def detach(self, observer: Observer) -> None:
        if observer in self._observers:
            self._observers.remove(observer)
    
    def notify(self) -> None:
        """Notify all observers of state change"""
        for observer in self._observers:
            observer.update(self)

# Concrete Observers
class EmailAlert(Observer):
    """Sends email when stock price changes"""
    
    def __init__(self, email: str, threshold: float):
        self.email = email
        self.threshold = threshold
    
    def update(self, subject: Stock) -> None:
        if subject.price > self.threshold:
            print(f"EMAIL to {self.email}: {subject.symbol} hit ${subject.price}")

class PriceLogger(Observer):
    """Logs all price changes"""
    
    def __init__(self):
        self.history = []
    
    def update(self, subject: Stock) -> None:
        entry = f"{subject.symbol}: ${subject.price}"
        self.history.append(entry)
        print(f"LOG: {entry}")

class TradingBot(Observer):
    """Executes trades based on price changes"""
    
    def __init__(self, buy_threshold: float, sell_threshold: float):
        self.buy_threshold = buy_threshold
        self.sell_threshold = sell_threshold
    
    def update(self, subject: Stock) -> None:
        if subject.price < self.buy_threshold:
            print(f"BOT: Buying {subject.symbol} at ${subject.price}")
        elif subject.price > self.sell_threshold:
            print(f"BOT: Selling {subject.symbol} at ${subject.price}")

# Test the Observer pattern
stock = Stock("AAPL", 150.0)

# Create observers
email = EmailAlert("investor@example.com", threshold=160.0)
logger = PriceLogger()
bot = TradingBot(buy_threshold=140.0, sell_threshold=170.0)

# Attach observers
stock.attach(email)
stock.attach(logger)
stock.attach(bot)

print("\n--- Price changes ---")
stock.price = 155.0  # Logged
stock.price = 165.0  # Logged + Email alert
stock.price = 135.0  # Logged + Bot buys

assert len(logger.history) == 3
print("\n✓ Observer pattern test passed!")

**Observer Pattern Benefits:**
- Loose coupling between subject and observers
- Dynamic relationships (add/remove at runtime)
- Open/Closed Principle (add new observers without modifying subject)
- Broadcast communication

**Drawbacks:**
- Can cause memory leaks if observers not properly removed
- Notification order not guaranteed
- Can have performance issues with many observers

**Real-World Usage:**
- Event handling systems (GUI, DOM)
- Pub/Sub messaging
- Model-View-Controller (MVC)
- Reactive programming (RxJS, RxPy)

### Problem 2: Factory Pattern (Medium)

**Problem Statement:**
Implement a Factory pattern for creating different types of notifications.

**Requirements:**
- Support multiple notification types (Email, SMS, Push)
- Each type has different implementation
- Factory decides which class to instantiate
- Easy to add new notification types

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Type

# Product interface
class Notification(ABC):
    """Abstract notification interface"""
    
    @abstractmethod
    def send(self, recipient: str, message: str) -> bool:
        """Send notification to recipient"""
        pass

# Concrete Products
class EmailNotification(Notification):
    """Email notification implementation"""
    
    def send(self, recipient: str, message: str) -> bool:
        print(f"📧 Sending email to {recipient}: {message}")
        # Actual email sending logic here
        return True

class SMSNotification(Notification):
    """SMS notification implementation"""
    
    def send(self, recipient: str, message: str) -> bool:
        print(f"📱 Sending SMS to {recipient}: {message}")
        # Actual SMS sending logic here
        return True

class PushNotification(Notification):
    """Push notification implementation"""
    
    def send(self, recipient: str, message: str) -> bool:
        print(f"🔔 Sending push to {recipient}: {message}")
        # Actual push notification logic here
        return True

# Factory
class NotificationFactory:
    """Factory for creating notifications"""
    
    # Registry of notification types
    _creators: Dict[str, Type[Notification]] = {}
    
    @classmethod
    def register(cls, notification_type: str, creator: Type[Notification]) -> None:
        """Register a new notification type"""
        cls._creators[notification_type] = creator
    
    @classmethod
    def create(cls, notification_type: str) -> Notification:
        """Create a notification of the specified type"""
        creator = cls._creators.get(notification_type)
        if not creator:
            raise ValueError(f"Unknown notification type: {notification_type}")
        return creator()
    
    @classmethod
    def get_available_types(cls) -> list:
        """Get list of registered notification types"""
        return list(cls._creators.keys())

# Register notification types
NotificationFactory.register('email', EmailNotification)
NotificationFactory.register('sms', SMSNotification)
NotificationFactory.register('push', PushNotification)

# Usage
def send_notification(notification_type: str, recipient: str, message: str):
    """Send notification using factory"""
    notification = NotificationFactory.create(notification_type)
    return notification.send(recipient, message)

# Test the factory
print("Available types:", NotificationFactory.get_available_types())
print()

send_notification('email', 'user@example.com', 'Hello from Email')
send_notification('sms', '+1234567890', 'Hello from SMS')
send_notification('push', 'device_id_123', 'Hello from Push')

# Test error handling
try:
    send_notification('fax', 'someone', 'Hello')  # Unknown type
except ValueError as e:
    print(f"\n✓ Error handling works: {e}")

print("✓ Factory pattern test passed!")

**Factory Pattern Variations:**

1. **Simple Factory** (above):
   - Single factory class
   - Static/class methods
   - Simple to implement

2. **Factory Method:**
   - Each concrete factory subclass
   - Override factory method
   - More flexible

3. **Abstract Factory:**
   - Families of related objects
   - More complex
   - Used in UI frameworks

**Benefits:**
- Encapsulates object creation
- Easy to add new types
- Single Responsibility Principle
- Client doesn't need to know concrete classes

**Follow-up Questions:**
- How to handle dependencies for each notification type?
- How to add configuration/credentials?
- When to use Factory vs Builder vs Prototype?

### Problem 3: Strategy Pattern (Medium)

**Problem Statement:**
Implement a payment processing system that supports multiple payment strategies.

**Requirements:**
- Support different payment methods (Credit Card, PayPal, Bitcoin)
- Each method has different validation and processing logic
- Client can switch payment method at runtime

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Strategy interface
class PaymentStrategy(ABC):
    """Abstract payment strategy"""
    
    @abstractmethod
    def validate(self) -> bool:
        """Validate payment details"""
        pass
    
    @abstractmethod
    def process_payment(self, amount: float) -> bool:
        """Process the payment"""
        pass

# Concrete Strategies
class CreditCardPayment(PaymentStrategy):
    """Credit card payment strategy"""
    
    def __init__(self, card_number: str, cvv: str, expiry: str):
        self.card_number = card_number
        self.cvv = cvv
        self.expiry = expiry
    
    def validate(self) -> bool:
        """Validate card details"""
        # Simple validation (real implementation would be more complex)
        valid_card = len(self.card_number) == 16 and self.card_number.isdigit()
        valid_cvv = len(self.cvv) == 3 and self.cvv.isdigit()
        return valid_card and valid_cvv
    
    def process_payment(self, amount: float) -> bool:
        if not self.validate():
            print("❌ Invalid credit card details")
            return False
        
        print(f"💳 Processing ${amount} via Credit Card ending in {self.card_number[-4:]}")
        # Actual payment processing here
        return True

class PayPalPayment(PaymentStrategy):
    """PayPal payment strategy"""
    
    def __init__(self, email: str, password: str):
        self.email = email
        self.password = password
    
    def validate(self) -> bool:
        """Validate PayPal credentials"""
        return '@' in self.email and len(self.password) > 0
    
    def process_payment(self, amount: float) -> bool:
        if not self.validate():
            print("❌ Invalid PayPal credentials")
            return False
        
        print(f"🅿️ Processing ${amount} via PayPal account {self.email}")
        # Actual PayPal API call here
        return True

class BitcoinPayment(PaymentStrategy):
    """Bitcoin payment strategy"""
    
    def __init__(self, wallet_address: str):
        self.wallet_address = wallet_address
    
    def validate(self) -> bool:
        """Validate Bitcoin wallet address"""
        # Simplified validation
        return len(self.wallet_address) > 20
    
    def process_payment(self, amount: float) -> bool:
        if not self.validate():
            print("❌ Invalid Bitcoin wallet address")
            return False
        
        print(f"₿ Processing ${amount} via Bitcoin to {self.wallet_address[:10]}...")
        # Actual blockchain transaction here
        return True

# Context
class ShoppingCart:
    """Shopping cart that uses payment strategy"""
    
    def __init__(self):
        self.items = []
        self.payment_strategy: Optional[PaymentStrategy] = None
    
    def add_item(self, item: str, price: float):
        """Add item to cart"""
        self.items.append({'item': item, 'price': price})
    
    def get_total(self) -> float:
        """Calculate total price"""
        return sum(item['price'] for item in self.items)
    
    def set_payment_strategy(self, strategy: PaymentStrategy):
        """Set payment method (strategy)"""
        self.payment_strategy = strategy
    
    def checkout(self) -> bool:
        """Process checkout using current strategy"""
        if not self.payment_strategy:
            print("❌ No payment method selected")
            return False
        
        total = self.get_total()
        print(f"\nTotal: ${total}")
        return self.payment_strategy.process_payment(total)

# Test the Strategy pattern
cart = ShoppingCart()
cart.add_item("Laptop", 999.99)
cart.add_item("Mouse", 29.99)

# Try different payment strategies
print("--- Paying with Credit Card ---")
cart.set_payment_strategy(CreditCardPayment("1234567890123456", "123", "12/25"))
assert cart.checkout() == True

print("\n--- Paying with PayPal ---")
cart.set_payment_strategy(PayPalPayment("user@example.com", "secret"))
assert cart.checkout() == True

print("\n--- Paying with Bitcoin ---")
cart.set_payment_strategy(BitcoinPayment("1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa"))
assert cart.checkout() == True

print("\n✓ Strategy pattern test passed!")

**Strategy Pattern Benefits:**
- Encapsulates algorithms/behaviors
- Runtime algorithm selection
- Eliminates conditional statements
- Open/Closed Principle

**Strategy vs State Pattern:**
- **Strategy:** Client chooses strategy
- **State:** Context transitions between states automatically

**Real-World Usage:**
- Sorting algorithms (Timsort, Quicksort)
- Compression algorithms (ZIP, RAR, 7z)
- Route planning (fastest, shortest, scenic)
- Payment processing

**Follow-up Questions:**
- How to add fees/taxes specific to each payment method?
- How to implement refunds?
- When to use Strategy vs Command pattern?

<a id='oop-design'></a>
## 2. OOP Design Problems

### Problem 4: Design a Parking Lot System (Medium)

**Problem Statement:**
Design a parking lot system that can:
- Handle different vehicle types (Car, Motorcycle, Truck)
- Different spot sizes (Compact, Regular, Large)
- Track occupancy and availability
- Calculate parking fees based on time

**Requirements:**
- Park a vehicle
- Remove a vehicle
- Get available spots
- Calculate fees

In [ ]:
from abc import ABC, abstractmethod
from enum import Enum
from datetime import datetime, timedelta
from typing import Optional, List

# Enums
class VehicleType(Enum):
    MOTORCYCLE = 1
    CAR = 2
    TRUCK = 3

class SpotSize(Enum):
    COMPACT = 1
    REGULAR = 2
    LARGE = 3

# Vehicle classes
class Vehicle(ABC):
    """Abstract vehicle class"""
    
    def __init__(self, license_plate: str):
        self.license_plate = license_plate
        self.entry_time: Optional[datetime] = None
    
    @abstractmethod
    def get_type(self) -> VehicleType:
        pass
    
    @abstractmethod
    def get_required_spot_size(self) -> SpotSize:
        pass

class Motorcycle(Vehicle):
    def get_type(self) -> VehicleType:
        return VehicleType.MOTORCYCLE
    
    def get_required_spot_size(self) -> SpotSize:
        return SpotSize.COMPACT

class Car(Vehicle):
    def get_type(self) -> VehicleType:
        return VehicleType.CAR
    
    def get_required_spot_size(self) -> SpotSize:
        return SpotSize.REGULAR

class Truck(Vehicle):
    def get_type(self) -> VehicleType:
        return VehicleType.TRUCK
    
    def get_required_spot_size(self) -> SpotSize:
        return SpotSize.LARGE

# Parking spot
class ParkingSpot:
    """Represents a single parking spot"""
    
    def __init__(self, spot_id: int, size: SpotSize):
        self.spot_id = spot_id
        self.size = size
        self.vehicle: Optional[Vehicle] = None
    
    def is_available(self) -> bool:
        return self.vehicle is None
    
    def can_fit_vehicle(self, vehicle: Vehicle) -> bool:
        """Check if vehicle can fit in this spot"""
        return (self.is_available() and 
                self.size.value >= vehicle.get_required_spot_size().value)
    
    def park_vehicle(self, vehicle: Vehicle) -> bool:
        """Park a vehicle in this spot"""
        if not self.can_fit_vehicle(vehicle):
            return False
        
        self.vehicle = vehicle
        vehicle.entry_time = datetime.now()
        return True
    
    def remove_vehicle(self) -> Optional[Vehicle]:
        """Remove and return the parked vehicle"""
        vehicle = self.vehicle
        self.vehicle = None
        return vehicle

# Parking lot
class ParkingLot:
    """Main parking lot system"""
    
    HOURLY_RATE = {
        VehicleType.MOTORCYCLE: 2.0,
        VehicleType.CAR: 5.0,
        VehicleType.TRUCK: 10.0
    }
    
    def __init__(self):
        self.spots: List[ParkingSpot] = []
        self.vehicle_to_spot = {}  # license_plate -> spot_id
    
    def add_spot(self, spot: ParkingSpot):
        """Add a parking spot"""
        self.spots.append(spot)
    
    def find_available_spot(self, vehicle: Vehicle) -> Optional[ParkingSpot]:
        """Find first available spot for vehicle"""
        for spot in self.spots:
            if spot.can_fit_vehicle(vehicle):
                return spot
        return None
    
    def park_vehicle(self, vehicle: Vehicle) -> bool:
        """Park a vehicle in the lot"""
        # Check if already parked
        if vehicle.license_plate in self.vehicle_to_spot:
            print(f"Vehicle {vehicle.license_plate} is already parked")
            return False
        
        # Find available spot
        spot = self.find_available_spot(vehicle)
        if not spot:
            print(f"No available spots for {vehicle.get_type().name}")
            return False
        
        # Park the vehicle
        if spot.park_vehicle(vehicle):
            self.vehicle_to_spot[vehicle.license_plate] = spot.spot_id
            print(f"✓ Parked {vehicle.license_plate} in spot {spot.spot_id}")
            return True
        
        return False
    
    def remove_vehicle(self, license_plate: str) -> Optional[float]:
        """Remove vehicle and return fee"""
        if license_plate not in self.vehicle_to_spot:
            print(f"Vehicle {license_plate} not found")
            return None
        
        # Find and remove vehicle
        spot_id = self.vehicle_to_spot[license_plate]
        spot = next(s for s in self.spots if s.spot_id == spot_id)
        vehicle = spot.remove_vehicle()
        
        del self.vehicle_to_spot[license_plate]
        
        # Calculate fee
        fee = self.calculate_fee(vehicle)
        print(f"✓ Removed {license_plate} from spot {spot_id}. Fee: ${fee:.2f}")
        return fee
    
    def calculate_fee(self, vehicle: Vehicle) -> float:
        """Calculate parking fee based on time"""
        if not vehicle.entry_time:
            return 0.0
        
        duration = datetime.now() - vehicle.entry_time
        hours = duration.total_seconds() / 3600
        # Minimum 1 hour
        hours = max(1, hours)
        
        rate = self.HOURLY_RATE[vehicle.get_type()]
        return hours * rate
    
    def get_available_spots_count(self) -> dict:
        """Get count of available spots by size"""
        counts = {size: 0 for size in SpotSize}
        for spot in self.spots:
            if spot.is_available():
                counts[spot.size] += 1
        return counts

# Test the parking lot system
parking_lot = ParkingLot()

# Add spots
for i in range(5):
    parking_lot.add_spot(ParkingSpot(i, SpotSize.COMPACT))
for i in range(5, 15):
    parking_lot.add_spot(ParkingSpot(i, SpotSize.REGULAR))
for i in range(15, 20):
    parking_lot.add_spot(ParkingSpot(i, SpotSize.LARGE))

print("Initial available spots:", parking_lot.get_available_spots_count())
print()

# Park vehicles
motorcycle = Motorcycle("MC-123")
car = Car("CAR-456")
truck = Truck("TRK-789")

parking_lot.park_vehicle(motorcycle)
parking_lot.park_vehicle(car)
parking_lot.park_vehicle(truck)

print("\nAfter parking:", parking_lot.get_available_spots_count())

# Remove vehicles
print()
parking_lot.remove_vehicle("CAR-456")
parking_lot.remove_vehicle("MC-123")

print("\n✓ Parking lot system test passed!")

**OOP Design Principles Applied:**

1. **Encapsulation:** Each class manages its own data
2. **Abstraction:** Vehicle is abstract base class
3. **Inheritance:** Specific vehicle types inherit from Vehicle
4. **Polymorphism:** Different vehicles behave differently

**SOLID Principles:**
- **S**ingle Responsibility: Each class has one job
- **O**pen/Closed: Easy to add new vehicle types
- **L**iskov Substitution: Can use any Vehicle subclass
- **I**nterface Segregation: Small, focused interfaces
- **D**ependency Inversion: Depend on abstractions

**Follow-up Questions:**
- How to add floors to the parking lot?
- How to implement reserved parking?
- How to handle multiple entrances/exits?
- How to add different pricing tiers?

## Summary

### Design Patterns Covered

1. **Observer Pattern**
   - One-to-many dependency
   - Event handling
   - Use case: Stock monitoring, GUI events

2. **Factory Pattern**
   - Object creation abstraction
   - Registry pattern
   - Use case: Notifications, document creation

3. **Strategy Pattern**
   - Interchangeable algorithms
   - Runtime selection
   - Use case: Payment methods, sorting

### OOP Design Skills

- **Requirement Analysis:** Understanding problem domain
- **Class Design:** Proper abstraction and hierarchy
- **Interface Design:** Clean APIs
- **Error Handling:** Validation and edge cases
- **Extensibility:** Easy to add new features

### Interview Tips

1. **Clarify Requirements:**
   - Ask about scale
   - Identify constraints
   - Discuss assumptions

2. **Start Simple:**
   - Basic functionality first
   - Iterate and improve
   - Don't over-engineer

3. **Explain Trade-offs:**
   - Discuss alternatives
   - Mention pros/cons
   - Consider future changes

4. **Use Diagrams:**
   - Class diagrams
   - Sequence diagrams
   - Component diagrams

### Common Design Patterns

**Creational:**
- Factory, Abstract Factory
- Builder
- Singleton
- Prototype

**Structural:**
- Adapter
- Decorator
- Facade
- Proxy

**Behavioral:**
- Observer
- Strategy
- Command
- Iterator
- State

### Additional Practice Problems

1. **Design an Elevator System**
2. **Design a Library Management System**
3. **Design a Chess Game**
4. **Design a Vending Machine**
5. **Design a Hotel Reservation System**
6. **Design a Movie Ticket Booking System**
7. **Design a Social Media Feed**
8. **Design a File System**

### Resources

- [Design Patterns: Elements of Reusable Object-Oriented Software](https://en.wikipedia.org/wiki/Design_Patterns) (Gang of Four)
- [Refactoring Guru](https://refactoring.guru/design-patterns) - Design patterns
- [SOLID Principles](https://en.wikipedia.org/wiki/SOLID)
- [System Design Primer](https://github.com/donnemartin/system-design-primer)